In [40]:
import PyPDF2
import re
import os

In [33]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        pdf = PyPDF2.PdfReader(f)
        text = '\n'.join([pdf.pages[page_num].extract_text() for page_num in range(len(pdf.pages))])
        return text

In [34]:
def get_directory_files(dir_path):
    files = []
    for file_path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, file_path)) and (".pdf" in file_path):
            files.append(file_path)

    return files

In [67]:
def get_student_name(file_name):
    name = file_name.split(' ')
    first_name = name[0].split('_')[1]
    last_name = name[1].split('_')[0]
    return first_name, last_name

In [36]:
def all_topological_sorts(graph):
    nodes = list(graph.keys())
    num_nodes = len(nodes)

    in_degree = {node: 0 for node in nodes}

    sorted_list = []
    all_sorts = []  

    visited = {node: False for node in nodes}

    for node in graph:
        for adj in graph[node]:
            in_degree[adj] += 1

    def dfs():
        is_all_visited = True

        for v in nodes:
            if in_degree[v] == 0 and not visited[v]:
                for adj in graph[v]:
                    in_degree[adj] -= 1

                sorted_list.append(v)
                visited[v] = True
                dfs()  


                visited[v] = False
                sorted_list.remove(v)
                for adj in graph[v]:
                    in_degree[adj] += 1
                is_all_visited = False

        if is_all_visited:
            all_sorts.append(sorted_list.copy())  

    dfs()
    return all_sorts  


graph = {
    'A': [],
    'B': ['A', 'C', 'D', 'E'],
    'C': ['D'],
    'D': ['H'],
    'E': [],
    'F': ['D', 'E'],
    'G': ['E', 'F', 'H'],
    'H': []
}


result = all_topological_sorts(graph)
print(len(result))  

132


In [61]:


def generate_regex(char_list):
    pattern = ''
    for char in char_list:
        if char in '.^$*+?{}[]\\|()':
            char = '\\' + char
        pattern += f"(?=.*[{re.escape(char)}])"

    return pattern

count = 0
for topological_sort in result:

    regex = generate_regex(topological_sort)

    pdf_path = '/Users/mahmood/Desktop/grading/week11/11/files/files_Adel Gaznanov_374141_assignsubmission_file__AdelGaznanov_problem_set_11.pdf'
    test_string = extract_text_from_pdf(pdf_path)
    match = re.search(regex, test_string, re.DOTALL)
    if bool(match) == True:
        count += 1
print (count)


132


In [62]:
def grade(submission, results):
    text = extract_text_from_pdf("/Users/mahmood/Desktop/grading/week11/11/files/" + str(submission))
    first_name, last_name = get_student_name(str(submission))
    count = 0
    for topological_sort in result:

        regex = generate_regex(topological_sort)
        
        match = re.search(regex, text, re.DOTALL)
        if bool(match) == True:
            count += 1
        
    return first_name, last_name, count

In [65]:
def grader(path):
    path_files = get_directory_files(path)
    for file in path_files:
        first, last, cout = grade(file, result)
        print(first, last, cout)

In [68]:
grader('/Users/mahmood/Desktop/grading/week11/11/files')

Sergey Aitov 132
Klimentii Chistyakov 132
Vazir Vadirov 132
Anton Korotkov 132
Saidaziz Kadirov 132
Sofia Goryunova 132
Ilyas Khatipov 132
Janna Ivanova 132
Rail Sabirov 132
Ilya Overin 132
Maria Rokkel 132
Nikita Menshikov 132
Aleksei Fominykh 132
Timur Usmanov 132
Egor Torshin 132
Elisa Caruso 132
Egor Sergeev 132
Nikita Shavrin 132
Marsel Fayzullin 132
Kostya Zimin 132
Samat Iakupov 132
Danil Eramasov 132
Ivan Savelev 132
Olesia Novoselova 132
Ivan Chabanov 132
Daria Nikolaeva 132
Mariia Nikolashina 132
Denis Troegubov 132
Elina Kuzmichyova 132
Ilya Maksimov 132
Nikita Zagainov 132
Aleksandr Gavkovskii 132
Timur Salakhov 132
Aleksandr Mikhailov 132
Artyom Grishin 132
Maxim Fomin 132
Dmitry Korletyanu 132
Daniil Tskhe 132
Lana Ermolaeva 132
Dziyana Melnikava 132
Ulyana Chaikouskaya 132
Daniil Mayorov 132
Ivan Lobazov 132
Nikita Timofeev 132
Sarmat Lutfullin 132
Alina Suhoverkova 132
Timur Farizunov 132
Aliia Bashirova 132
Maksim Menshikh 132
Aleksandr Andreev 132
Oleg Shchendrigin 13

In [59]:
result

[['B', 'A', 'C', 'G', 'F', 'D', 'E', 'H'],
 ['B', 'A', 'C', 'G', 'F', 'D', 'H', 'E'],
 ['B', 'A', 'C', 'G', 'F', 'E', 'D', 'H'],
 ['B', 'A', 'G', 'C', 'F', 'D', 'E', 'H'],
 ['B', 'A', 'G', 'C', 'F', 'D', 'H', 'E'],
 ['B', 'A', 'G', 'C', 'F', 'E', 'D', 'H'],
 ['B', 'A', 'G', 'F', 'C', 'D', 'E', 'H'],
 ['B', 'A', 'G', 'F', 'C', 'D', 'H', 'E'],
 ['B', 'A', 'G', 'F', 'C', 'E', 'D', 'H'],
 ['B', 'A', 'G', 'F', 'E', 'C', 'D', 'H'],
 ['B', 'C', 'A', 'G', 'F', 'D', 'E', 'H'],
 ['B', 'C', 'A', 'G', 'F', 'D', 'H', 'E'],
 ['B', 'C', 'A', 'G', 'F', 'E', 'D', 'H'],
 ['B', 'C', 'G', 'A', 'F', 'D', 'E', 'H'],
 ['B', 'C', 'G', 'A', 'F', 'D', 'H', 'E'],
 ['B', 'C', 'G', 'A', 'F', 'E', 'D', 'H'],
 ['B', 'C', 'G', 'F', 'A', 'D', 'E', 'H'],
 ['B', 'C', 'G', 'F', 'A', 'D', 'H', 'E'],
 ['B', 'C', 'G', 'F', 'A', 'E', 'D', 'H'],
 ['B', 'C', 'G', 'F', 'D', 'A', 'E', 'H'],
 ['B', 'C', 'G', 'F', 'D', 'A', 'H', 'E'],
 ['B', 'C', 'G', 'F', 'D', 'E', 'A', 'H'],
 ['B', 'C', 'G', 'F', 'D', 'E', 'H', 'A'],
 ['B', 'C',